In [10]:
import re
import os
import shutil
import pandas as pd

# Borrar train folder

In [5]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [6]:
for di in os.listdir(path):
    shutil.rmtree(os.path.join(path, di, 'train'), ignore_errors=True)

# Borrar modelos malos

In [79]:
path = '/home/mr1142/Documents/Data/models/validation_results/validation_results' + '' + '.csv'
df = pd.read_csv(path)

In [80]:
df.sort_values('dice_coef_loss')

,type,name,loss,dice_coef_loss,loss_mask,accuracy,AUC,false_positives,false_negatives
28,unet,renacimiento_validation_1,0.032621,0.032179,0.092485,0.973949,0.974434,88665.0,80353.0
2,unet,_albumentation_validation_2.h5,0.034487,0.034487,0.095812,0.973940,0.974166,7962.0,5701.0
21,uloss,albumentation_rotate_3_validation_2,0.129951,0.034909,0.094568,0.971946,0.973499,87304.0,94711.0
20,uloss,albumentation_rotate_3_validation_1,0.128454,0.035279,0.092766,0.971818,0.972174,78499.0,104346.0
19,unet,albumentation_rotate_3_validation_2,0.035893,0.035434,0.097252,0.971598,0.971085,81626.0,102645.0
4,uloss,_albumentation_validation_2.h5,0.149065,0.036346,0.112719,0.972670,0.973263,6937.0,7392.0
3,unet,_albumentation_validation_3.h5,0.036912,0.036912,0.118864,0.972273,0.971054,7108.0,7429.0
17,unet,albumentation_rotate_3_validation_1,0.037508,0.036994,0.104480,0.970098,0.969842,87088.0,106918.0
0,unet,_albumentation_validation_1.h5,0.037296,0.037296,0.117240,0.971939,0.970628,7386.0,7326.0
23,uloss,validation_rotate_normalization_1,0.134512,0.037305,0.097394,0.970390,0.971013,91815.0,100297.0


In [81]:
malos = df[['type', 'name']][df.dice_coef_loss>0.05]

In [82]:
malos = [df['type'][i] + '_' + df['name'][i] for i in range(len(malos))]
malos = [re.sub('__', '_', mal) for mal in malos]
malos = [re.sub('.h5', '', mal) for mal in malos]

In [83]:
malos

[]

In [72]:
def eliminar(path, malos):
    files = os.listdir(path)
    files = [f for f in files if any([bool(re.search(mal, f)) for mal in malos])]
    if len(files) == len(malos):
        for file in files:
            try:
                os.remove(os.path.join(path, file))
            except:
                shutil.rmtree(os.path.join(path,file), ignore_errors=True)
    else:
        print('no tienen el mismo tamaño')

### Logs

In [71]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [74]:
eliminar(path, malos)

True

### Modelos

In [84]:
path = '/home/mr1142/Documents/Data/models'

In [ ]:
eliminar(path, malos)